In [44]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

spark = SparkSession.builder \
    .master('local') \
    .appName('recommendation') \
    .getOrCreate()

In [63]:
data_path = '/Users/hyunseokjung/data/spark_guide'

ratings = spark.read.text(f"{data_path}/sample_movielens_ratings.txt") \
    .rdd.toDF() \
    .selectExpr("split(value, '::') as col") \
    .selectExpr(
        "cast(col[0] as int) as userId",
        "cast(col[1] as int) as movieId",
        "cast(col[2] as float) as rating",
        "cast(col[3] as long) as timestamp"
    )

In [46]:
train,test = ratings.randomSplit([0.8, 0.2])

In [47]:
als = ALS() \
    .setMaxIter(5) \
    .setRegParam(0.01) \
    .setUserCol("userId") \
    .setItemCol("movieId") \
    .setRatingCol("rating")

In [48]:
print(als.explainParams())

alpha: alpha for implicit preference (default: 1.0)
blockSize: block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than remaining data in a partition then it is adjusted to the size of this data. (default: 4096)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False)
intermediateStorageLevel: StorageLevel for interme

In [64]:
alsModel = als.fit(train)
predictions = alsModel.transform(test)

In [66]:
predictions.show(3)

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|     0|      3|   1.0|1424380312| 0.6339553|
|     0|      5|   2.0|1424380312| 1.4654416|
|     0|     21|   1.0|1424380312| 1.6357095|
+------+-------+------+----------+----------+
only showing top 3 rows



In [74]:
alsModel.recommendForAllUsers(3) \
    .selectExpr("userId", "explode(recommendations)") \
    .show(6)
alsModel.recommendForAllItems(3) \
    .selectExpr("movieId", "explode(recommendations)") \
    .show(6)

+------+---------------+
|userId|            col|
+------+---------------+
|    20|{22, 4.6268606}|
|    20| {75, 4.272371}|
|    20|{54, 4.1329565}|
|    10| {38, 4.204379}|
|    10|{46, 3.8890886}|
|    10| {74, 3.536286}|
+------+---------------+
only showing top 6 rows



+-------+---------------+
|movieId|            col|
+-------+---------------+
|     20|{26, 4.9450316}|
|     20| {17, 4.641791}|
|     20| {2, 4.1928797}|
|     40|  {6, 4.387648}|
|     40| {2, 3.8655288}|
|     40| {7, 3.4234028}|
+-------+---------------+
only showing top 6 rows



In [77]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator() \
    .setMetricName("rmse") \
    .setLabelCol("rating") \
    .setPredictionCol("prediction")

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

Root-mean-square error = 2.019469736413673


In [78]:
from pyspark.mllib.evaluation import RegressionMetrics

regComparison = predictions.select("rating", "prediction") \
    .rdd.map(lambda x: (x(0), x(1)))
metrics = RegressionMetrics(regComparison)

/Users/hyunseokjung/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [86]:
metrics.rootMeanSquaredError

22/12/02 13:54:15 ERROR Executor: Exception in task 0.0 in stage 715.0 (TID 1842)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/Users/hyunseokjung/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/session.py", line 910, in prepare
    verify_func(obj)
  File "/Users/hyunseokjung/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/types.py", line 1722, in verify
   

Py4JJavaError: An error occurred while calling o1333.rootMeanSquaredError.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 715.0 failed 1 times, most recent failure: Lost task 0.0 in stage 715.0 (TID 1842) (ingu627 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/hyunseokjung/spark-3.3.0/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
  File "/Users/hyunseokjung/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/s

In [79]:
from pyspark.mllib.evaluation import RankingMetrics, RegressionMetrics
from pyspark.sql.functions import col, expr

perUserActual = predictions \
    .where("rating > 2.5") \
    .groupBy("userId") \
    .agg(expr("collect_set(movieId) as movies"))

In [55]:
perUserPredictions = predictions \
    .orderBy(col("userId"), expr("prediction DESC")) \
    .groupBy("userId") \
    .agg(expr("collect_list(movieId) as movies"))

In [56]:
perUserActualvPred = perUserActual.join(perUserPredictions, ["userId"]).rdd \
    .map(lambda row: (row[1], row[2][:15]))
ranks = RankingMetrics(perUserActualvPred)

/Users/hyunseokjung/opt/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [57]:
ranks.meanAveragePrecision
ranks.precisionAt(5)

0.5076923076923078